<a href="https://colab.research.google.com/github/ounospanas/KD_PULSE/blob/main/notebooks/load_segment_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download ppg-dalia

In [2]:
!wget https://archive.ics.uci.edu/static/public/495/ppg+dalia.zip
!unzip ppg+dalia.zip

--2024-01-24 08:53:30--  https://archive.ics.uci.edu/static/public/495/ppg+dalia.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘ppg+dalia.zip’

ppg+dalia.zip           [          <=>       ]   2.67G  46.5MB/s    in 60s     

2024-01-24 08:54:30 (45.7 MB/s) - ‘ppg+dalia.zip’ saved [2865111320]



In [6]:
!unzip data.zip

Archive:  data.zip
   creating: PPG_FieldStudy/
  inflating: PPG_FieldStudy/PPG_FieldStudy_readme.pdf  
   creating: PPG_FieldStudy/S1/
   creating: PPG_FieldStudy/S10/
  inflating: PPG_FieldStudy/S10/S10.pkl  
  inflating: PPG_FieldStudy/S10/S10_activity.csv  
  inflating: PPG_FieldStudy/S10/S10_E4.zip  
  inflating: PPG_FieldStudy/S10/S10_quest.csv  
  inflating: PPG_FieldStudy/S10/S10_RespiBAN.h5  
   creating: PPG_FieldStudy/S11/
  inflating: PPG_FieldStudy/S11/S11.pkl  
  inflating: PPG_FieldStudy/S11/S11_activity.csv  
  inflating: PPG_FieldStudy/S11/S11_E4.zip  
  inflating: PPG_FieldStudy/S11/S11_quest.csv  
  inflating: PPG_FieldStudy/S11/S11_RespiBAN.h5  
   creating: PPG_FieldStudy/S12/
  inflating: PPG_FieldStudy/S12/S12.pkl  
  inflating: PPG_FieldStudy/S12/S12_activity.csv  
  inflating: PPG_FieldStudy/S12/S12_E4.zip  
  inflating: PPG_FieldStudy/S12/S12_quest.csv  
  inflating: PPG_FieldStudy/S12/S12_RespiBAN.h5  
   creating: PPG_FieldStudy/S13/
  inflating: PPG_FieldSt

In [1]:
import pickle
import random
import os
import numpy as np
from skimage.util.shape import view_as_windows
from scipy.io import loadmat

In [7]:
cf = 'PPG_FieldStudy/'

In [8]:
"""
Reusing code from https://github.com/eml-eda/q-ppg
"""
def preprocessing(cf):
    # Sampling frequency of acceleration data in PPG_Dalia dataset
    # The sampling frequency of ppg data in PPG_Dalia dataset is fs_PPG_Dalia*2
    fs_PPG_Dalia = 32
    fs_eda_temp = 4
    window = 8
    move = 2

    fs_activity = 4

    Session = dict()
    S = dict()
    acc = dict()
    ppg = dict()
    activity = dict()
    eda = dict()
    temp = dict()


    random.seed(20)

    ground_truth = dict()

    #val = dataset


    numbers= list(range(1,16))
    session_list=random.sample(numbers,len(numbers))
    for j in session_list:
        paz = j

        with open(cf + 'S' + str(j) +'/S' + str(j) +'.pkl', 'rb') as f:
            S[paz] = pickle.load(f, encoding='latin1')
        ppg[paz] = S[paz]['signal']['wrist']['BVP'][::2]
        acc[paz] = S[paz]['signal']['wrist']['ACC']
        activity[paz] = S[paz]['activity']
        ground_truth[paz] = S[paz]['label']
        eda[j] = S[j]['signal']['wrist']['EDA']
        temp[j] = S[j]['signal']['wrist']['TEMP']
        print("I read all", j)

    sig = dict()
    act_list = []
    groups= []
    sig_list = []
    ground_truth_list = []
    bio = dict()
    bio_list = []


    # Loop on keys of dictionary ground_truth
    for k in ground_truth:
        # Remeber to set the desired time window
        bio[k] = np.concatenate((eda[k],temp[k]),axis=1)
        bio[k]= np.moveaxis(view_as_windows(bio[k], (fs_eda_temp*window,2),int(fs_eda_temp*move))[:,0,:,:],1,2)
        activity[k] = np.moveaxis(view_as_windows(activity[k], (4*8,1),4*2)[:,0,:,:],1,2)
        activity[k] = activity[k][:,:,0]
        sig[k] = np.concatenate((ppg[k],acc[k]),axis=1)
        sig[k]= np.moveaxis(view_as_windows(sig[k], (fs_PPG_Dalia*8,4),fs_PPG_Dalia*2)[:,0,:,:],1,2)
        groups.append(np.full(sig[k].shape[0],k))
        bio_list.append(bio[k])
        sig_list.append(sig[k])
        act_list.append(activity[k])
        ground_truth[k] = np.reshape(ground_truth[k], (ground_truth[k].shape[0],1))
        ground_truth_list.append(ground_truth[k])
        print('I finished',k)

    #print("gruppo",groups)
    groups = np.hstack(groups)
    X = np.vstack(sig_list)
    y = np.reshape(np.vstack(ground_truth_list),(-1,1))
    bio = np.vstack(bio_list)

    act = np.vstack(act_list)

    data = dict()
    data['X'] = X
    data['y'] = y
    data['groups'] = groups
    data['act'] = act
    data['bio'] = bio

    print("dimension data",X.shape, "dimesion labels", y.shape,"dimension groups",groups.shape)

    return X[:y.shape[0]], y, groups[:y.shape[0]], act[:y.shape[0]], bio[:y.shape[0]]

In [9]:
data = preprocessing(cf)

I read all 15
I read all 12
I read all 11
I read all 3
I read all 5
I read all 2
I read all 6
I read all 14
I read all 1
I read all 4
I read all 9
I read all 7
I read all 13
I read all 8
I read all 10
I finished 15
I finished 12
I finished 11
I finished 3
I finished 5
I finished 2
I finished 6
I finished 14
I finished 1
I finished 4
I finished 9
I finished 7
I finished 13
I finished 8
I finished 10
dimension data (64697, 4, 256) dimesion labels (64697, 1) dimension groups (64697,)


In [10]:
np.save('dalia_data.npy', data[0])
np.save('dalia_y.npy', data[1])
np.save('dalia_subs.npy', data[2])
np.save('dalia_acts.npy', data[3])